# Smart Gift Finder

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/robin-mongodb/mongochain/blob/main/examples/verticals/retail/single_agent_demo.ipynb)

An AI-powered gift recommendation agent that helps users find the perfect gift using semantic product search.

## Features Demonstrated
- **RAG (Retrieval-Augmented Generation)**: Vector search over product catalog
- **Semantic Product Search**: Find products matching vague, conversational queries
- **Multi-criteria Reasoning**: Intersect interests (e.g., "hiking" + "tech") with budget constraints
- **Memory Persistence**: Remember user preferences and past gift searches
- **Personalized Recommendations**: Context-aware gift suggestions with explanations

## Prerequisites
- MongoDB Atlas cluster
- Voyage AI API key
- OpenAI API key


## 1. Setup


In [ ]:
# Install mongochain from GitHub
%pip install -q git+https://github.com/robin-mongodb/mongochain.git

In [ ]:
from getpass import getpass

# API Keys (input hidden)
MONGO_URI = getpass("MongoDB URI: ")
VOYAGE_API_KEY = getpass("Voyage API Key: ")
OPENAI_API_KEY = getpass("OpenAI API Key: ")

## 2. Seed Product Catalog

Create a diverse product catalog with rich descriptions. Each product includes:
- Name and category
- Detailed description with features and use cases
- Price and tags for filtering
- Vector embedding for semantic search


In [ ]:
from pymongo import MongoClient
import voyageai

# User ID for this demo
USER_ID = "gift_shopper@email.com"

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client["smart_gift_finder"]  # Same as agent name
products = db["product_catalog"]

# Initialize Voyage for embeddings
voyage = voyageai.Client(api_key=VOYAGE_API_KEY)

# Clear existing product data
products.delete_many({})


In [ ]:
# Product catalog with diverse items spanning multiple interests
PRODUCT_CATALOG = [
    # Tech + Outdoor
    {
        "name": "SolarPower Pro 20000",
        "category": "Electronics",
        "price": 45.99,
        "description": "Rugged solar power bank with 20000mAh capacity. Features dual USB-C ports, LED flashlight, and waterproof design. Perfect for hiking, camping, and outdoor adventures. Charges phones up to 5 times. Built-in carabiner clip for easy attachment to backpacks.",
        "tags": ["tech", "outdoor", "hiking", "camping", "sustainable", "travel"]
    },
    {
        "name": "HydroSmart Temperature Bottle",
        "category": "Outdoor Gear",
        "price": 39.99,
        "description": "Smart water bottle with LED temperature display and hydration reminders. Double-walled vacuum insulation keeps drinks cold for 24 hours or hot for 12 hours. Syncs with fitness apps via Bluetooth. Great for gym, hiking, or daily hydration tracking.",
        "tags": ["tech", "fitness", "hiking", "health", "smart home"]
    },
    {
        "name": "TrailTech GPS Watch",
        "category": "Wearables",
        "price": 129.99,
        "description": "Adventure GPS smartwatch with topographic maps, altimeter, barometer, and compass. 14-day battery life in GPS mode. Tracks hiking, running, cycling, and swimming. Features emergency SOS and weather alerts. Perfect for serious outdoor enthusiasts.",
        "tags": ["tech", "outdoor", "hiking", "fitness", "adventure", "wearable"]
    },
    {
        "name": "Compact Trail Speaker",
        "category": "Electronics",
        "price": 34.99,
        "description": "Ultra-portable Bluetooth speaker with IPX7 waterproof rating. 12-hour battery, carabiner clip, and 360-degree sound. Floats in water. Ideal for hiking, beach, pool parties, and outdoor adventures.",
        "tags": ["tech", "outdoor", "music", "hiking", "beach", "travel"]
    },
    
    # Home + Cooking
    {
        "name": "ChefMate Smart Thermometer",
        "category": "Kitchen",
        "price": 49.99,
        "description": "Wireless meat thermometer with app-controlled temperature monitoring. Alerts your phone when food reaches target temp. Features 4 probes for multiple dishes. Perfect for grilling, smoking, and precision cooking.",
        "tags": ["tech", "cooking", "kitchen", "grilling", "smart home", "foodie"]
    },
    {
        "name": "Artisan Pour-Over Set",
        "category": "Kitchen",
        "price": 42.00,
        "description": "Premium pour-over coffee brewing set with borosilicate glass carafe, stainless steel filter, and precision gooseneck kettle. Includes brewing guide for coffee enthusiasts. Makes cafe-quality coffee at home.",
        "tags": ["coffee", "kitchen", "artisan", "brewing", "gift set"]
    },
    {
        "name": "Smart Herb Garden",
        "category": "Home & Garden",
        "price": 89.99,
        "description": "Indoor smart garden with LED grow lights and automatic watering. Grow fresh herbs, lettuce, and tomatoes year-round. App controls light cycles and reminds you to add water. Includes 3 herb seed pods.",
        "tags": ["tech", "cooking", "garden", "sustainable", "smart home", "health"]
    },
    
    # Wellness + Self-Care
    {
        "name": "ZenGlow Aromatherapy Diffuser",
        "category": "Wellness",
        "price": 35.00,
        "description": "Elegant ultrasonic essential oil diffuser with color-changing LED lights and timer. Whisper-quiet operation for bedroom or office. Includes starter set of lavender, eucalyptus, and peppermint oils.",
        "tags": ["wellness", "relaxation", "self-care", "aromatherapy", "bedroom", "gift set"]
    },
    {
        "name": "Massage Gun Pro",
        "category": "Fitness",
        "price": 79.99,
        "description": "Deep tissue percussion massager with 6 speed settings and 4 interchangeable heads. Quiet brushless motor. Helps relieve muscle soreness after workouts, hiking, or long days. USB-C rechargeable with 6-hour battery.",
        "tags": ["fitness", "wellness", "recovery", "hiking", "sports", "self-care"]
    },
    {
        "name": "Sunrise Wake Light",
        "category": "Home",
        "price": 54.99,
        "description": "Sunrise simulation alarm clock that gradually brightens to wake you naturally. Features nature sounds, FM radio, and reading lamp. Improves morning mood and energy. Smart home compatible.",
        "tags": ["wellness", "sleep", "smart home", "bedroom", "relaxation"]
    },
    
    # Creative + Arts
    {
        "name": "Digital Sketch Tablet",
        "category": "Electronics",
        "price": 69.99,
        "description": "Pressure-sensitive drawing tablet with 8192 levels of sensitivity. Works with popular art software. Compact size perfect for digital art, photo editing, and note-taking. Great for artists and designers.",
        "tags": ["tech", "art", "creative", "drawing", "design", "digital"]
    },
    {
        "name": "Watercolor Travel Set",
        "category": "Art Supplies",
        "price": 38.00,
        "description": "Professional-grade portable watercolor set with 24 vibrant colors, travel brush, and mixing palette. Compact metal tin fits in a pocket. Perfect for outdoor painting, travel journaling, and plein air artists.",
        "tags": ["art", "creative", "travel", "painting", "outdoor", "journal"]
    },
    {
        "name": "Instant Photo Printer",
        "category": "Electronics",
        "price": 99.99,
        "description": "Portable Bluetooth photo printer for smartphones. Prints 2x3 inch photos with peel-and-stick backing. AR features bring photos to life. Perfect for scrapbooking, parties, and capturing memories.",
        "tags": ["tech", "photography", "creative", "memories", "party", "scrapbook"]
    },
    
    # Reading + Learning
    {
        "name": "BookLight Deluxe",
        "category": "Accessories",
        "price": 24.99,
        "description": "Rechargeable LED book light with warm/cool color modes and adjustable brightness. Clips onto books or stands on desk. Eye-friendly design for late-night reading. 40-hour battery life.",
        "tags": ["reading", "books", "bedroom", "travel", "study"]
    },
    {
        "name": "Language Learning Subscription",
        "category": "Digital",
        "price": 49.99,
        "description": "1-year premium subscription to language learning platform. Access 40+ languages with AI speech recognition, personalized lessons, and progress tracking. Perfect for travelers and lifelong learners.",
        "tags": ["education", "learning", "travel", "languages", "digital", "self-improvement"]
    },
    
    # Gaming + Fun
    {
        "name": "Retro Game Console",
        "category": "Gaming",
        "price": 59.99,
        "description": "Compact retro gaming console with 500+ classic games built-in. HDMI output and wireless controllers. Nostalgic 8-bit and 16-bit games. Perfect for game nights and reliving childhood memories.",
        "tags": ["gaming", "retro", "fun", "nostalgia", "party", "entertainment"]
    },
    {
        "name": "Strategy Board Game Collection",
        "category": "Games",
        "price": 44.99,
        "description": "Award-winning strategy board game with modular hex tiles and resource management. 2-6 players, 45-90 minute games. Highly replayable with expansion packs available. Great for game nights.",
        "tags": ["games", "board games", "strategy", "party", "family", "social"]
    },
    
    # Travel + Adventure
    {
        "name": "Travel Organizer Set",
        "category": "Travel",
        "price": 32.00,
        "description": "6-piece packing cube set with compression zippers. Includes shoe bag, toiletry pouch, and laundry bag. Keeps luggage organized and maximizes space. Essential for frequent travelers.",
        "tags": ["travel", "organization", "adventure", "practical", "luggage"]
    },
    {
        "name": "Adventure Journal",
        "category": "Stationery",
        "price": 28.00,
        "description": "Leather-bound travel journal with maps, trip planning pages, and archival-quality paper. Includes envelope pocket for tickets and memories. Perfect for documenting adventures and road trips.",
        "tags": ["travel", "journal", "writing", "adventure", "memories", "outdoor"]
    },
    {
        "name": "Neck Travel Pillow Premium",
        "category": "Travel",
        "price": 39.99,
        "description": "Memory foam travel pillow with 360-degree support. Includes sleep mask and earplugs. Compresses to 1/4 size for packing. Perfect for long flights, road trips, and camping.",
        "tags": ["travel", "comfort", "sleep", "airplane", "practical"]
    },
    
    # Pets
    {
        "name": "Smart Pet Feeder",
        "category": "Pet Supplies",
        "price": 69.99,
        "description": "Automatic pet food dispenser with app scheduling and portion control. Built-in camera to watch your pet eat. Voice recording to call pets at mealtime. Works with dry food.",
        "tags": ["tech", "pets", "smart home", "dogs", "cats", "practical"]
    },
    {
        "name": "GPS Pet Tracker",
        "category": "Pet Supplies",
        "price": 49.99,
        "description": "Lightweight GPS collar attachment for dogs and cats. Real-time location tracking, activity monitoring, and safe zone alerts. Waterproof with 7-day battery. Peace of mind for pet parents.",
        "tags": ["tech", "pets", "outdoor", "dogs", "cats", "safety"]
    },
    
    # Eco-Friendly
    {
        "name": "Beeswax Wrap Set",
        "category": "Kitchen",
        "price": 22.00,
        "description": "Reusable beeswax food wraps in 3 sizes. Natural alternative to plastic wrap. Organic cotton, beeswax, and jojoba oil. Washable and lasts 1 year. Perfect eco-conscious gift.",
        "tags": ["sustainable", "kitchen", "eco-friendly", "practical", "zero waste"]
    },
    {
        "name": "Bamboo Desk Organizer",
        "category": "Office",
        "price": 36.00,
        "description": "Sustainable bamboo desk organizer with phone stand, pen holder, and cable management. Minimalist design fits any workspace. Eco-friendly alternative to plastic organizers.",
        "tags": ["sustainable", "office", "organization", "work from home", "eco-friendly"]
    },
    {
        "name": "Solar Garden Lights Set",
        "category": "Outdoor",
        "price": 29.99,
        "description": "Set of 8 solar-powered LED garden path lights. Automatic dusk-to-dawn operation. Waterproof and wireless. Creates beautiful ambiance for gardens, patios, and walkways.",
        "tags": ["sustainable", "outdoor", "garden", "home", "solar", "eco-friendly"]
    }
]

print(f"📦 Preparing {len(PRODUCT_CATALOG)} products for catalog...")


In [ ]:
# Generate embeddings for all products
def create_product_text(product):
    """Create searchable text from product details."""
    return f"{product['name']}. {product['description']} Category: {product['category']}. Tags: {', '.join(product['tags'])}"

# Batch embed all products
product_texts = [create_product_text(p) for p in PRODUCT_CATALOG]
embeddings = voyage.embed(product_texts, model="voyage-3", input_type="document").embeddings

# Add embeddings to products and insert
for i, product in enumerate(PRODUCT_CATALOG):
    product["embedding"] = embeddings[i]
    product["searchable_text"] = product_texts[i]

products.insert_many(PRODUCT_CATALOG)
print(f"✅ Inserted {len(PRODUCT_CATALOG)} products into catalog")

# Show sample products
print("\nSample products:")
for p in PRODUCT_CATALOG[:5]:
    print(f"  ${p['price']:>6.2f} | {p['name']:<30} | {p['category']}")


In [ ]:
# Create vector search index for products
from pymongo.errors import OperationFailure

INDEX_NAME = "product_vector_index"

# Check if index exists
try:
    existing = list(products.list_search_indexes())
    index_exists = any(idx.get("name") == INDEX_NAME for idx in existing)
except OperationFailure:
    index_exists = False

if not index_exists:
    products.create_search_index({
        "definition": {
            "mappings": {
                "dynamic": True,
                "fields": {
                    "embedding": {
                        "type": "knnVector",
                        "dimensions": 1024,  # Voyage-3 dimensions
                        "similarity": "cosine"
                    }
                }
            }
        },
        "name": INDEX_NAME
    })
    print(f"✅ Vector index '{INDEX_NAME}' created (may take a few minutes to build)")
else:
    print(f"✅ Vector index '{INDEX_NAME}' already exists")


## 3. Create the Smart Gift Finder Agent

In [ ]:
from mongochain import MongoAgent

agent = MongoAgent(
    name="smart_gift_finder",
    persona="""You are a friendly and creative Gift Concierge with a talent for finding the perfect present.

Your approach:
- Ask clarifying questions to understand the recipient's interests and the occasion
- Think creatively about how different interests might intersect (e.g., "hiking" + "tech" = solar power bank)
- Always explain WHY a gift would be perfect for the recipient
- Consider budget constraints and offer options at different price points
- Remember details about past gift searches to provide personalized suggestions
- Be enthusiastic but not pushy — you genuinely want to help find something special

When recommending gifts:
1. Use the product search tool to find relevant items
2. Explain how each suggestion connects to the recipient's interests
3. Highlight unique features that make the gift thoughtful
4. Suggest gift combinations if appropriate""",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=OPENAI_API_KEY,
    llm_provider="openai"
)

print(f"\n{agent}")


## 4. Register Product Search Tool (RAG)

Create a semantic search tool that uses MongoDB Atlas Vector Search to find relevant products based on natural language queries.


In [ ]:
import voyageai
from pymongo import MongoClient

# Initialize clients for tool functions
_client = MongoClient(MONGO_URI)
_db = _client["smart_gift_finder"]
_products = _db["product_catalog"]
_voyage = voyageai.Client(api_key=VOYAGE_API_KEY)


def search_products(query: str, max_price: float = None, min_results: int = 5) -> str:
    """Search the product catalog using semantic vector search.
    
    Args:
        query: Natural language search query describing what you're looking for
        max_price: Maximum price filter (optional)
        min_results: Minimum number of results to return
    
    Returns:
        Formatted string with matching products and their details
    """
    # Generate query embedding
    query_embedding = _voyage.embed([query], model="voyage-3", input_type="query").embeddings[0]
    
    # Build vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "product_vector_index",
                "path": "embedding",
                "queryVector": query_embedding,
                "numCandidates": 50,
                "limit": min_results * 2  # Get extra to filter by price
            }
        },
        {
            "$project": {
                "name": 1,
                "category": 1,
                "price": 1,
                "description": 1,
                "tags": 1,
                "score": {"$meta": "vectorSearchScore"}
            }
        }
    ]
    
    # Add price filter if specified
    if max_price is not None:
        pipeline.append({"$match": {"price": {"$lte": max_price}}})
    
    pipeline.append({"$limit": min_results})
    
    try:
        results = list(_products.aggregate(pipeline))
    except Exception as e:
        # Fallback to text-based search if vector index not ready
        query_filter = {}
        if max_price:
            query_filter["price"] = {"$lte": max_price}
        results = list(_products.find(query_filter, {"embedding": 0}).limit(min_results))
    
    if not results:
        return f"No products found matching '{query}'. Try broader search terms."
    
    # Format results
    output = f"Found {len(results)} products matching '{query}':\n\n"
    
    for i, product in enumerate(results, 1):
        score = product.get('score', 0)
        relevance = "⭐⭐⭐" if score > 0.6 else "⭐⭐" if score > 0.4 else "⭐"
        
        output += f"{i}. {product['name']} — ${product['price']:.2f}\n"
        output += f"   Category: {product['category']} | Relevance: {relevance}\n"
        output += f"   {product['description'][:200]}...\n"
        output += f"   Tags: {', '.join(product.get('tags', [])[:5])}\n\n"
    
    return output


# Register the tool
agent.register_tool(
    name="search_products",
    func=search_products,
    description="Search the product catalog for gift ideas. Use natural language to describe interests, occasions, or gift types. Supports price filtering.",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Natural language search query (e.g., 'outdoor tech gadgets for hikers', 'creative gifts for artists')"
            },
            "max_price": {
                "type": "number",
                "description": "Maximum price in dollars (optional)"
            },
            "min_results": {
                "type": "integer",
                "description": "Minimum number of results to return (default: 5)"
            }
        },
        "required": ["query"]
    }
)

print("✅ Product search tool registered!")


## 5. Demo: Finding a Gift with Vague Requirements

Watch the agent use RAG to search the product catalog and reason about gift suggestions. The query mentions "hiking and tech" — the agent should find products that bridge both interests.


In [ ]:
# Initial gift request with vague but specific interests
response = agent.chat(
    user_id=USER_ID,
    message="Hi! I need help finding a gift for my friend Sarah. She loves hiking and is really into tech gadgets. My budget is around $50. Any ideas?"
)

print(response)


In [ ]:
# Follow-up with additional context
response = agent.chat(
    user_id=USER_ID,
    message="Those are great options! I think she'd really like something practical that she could use on her hiking trips. She mentioned her phone always dies when she's out on long trails."
)

print(response)


## 6. Demo: Different Gift Persona

Let's search for a completely different type of gift — this demonstrates the breadth of the catalog and the agent's ability to pivot.


# New gift search with different criteria
response = agent.chat(
    user_id=USER_ID,
    message="Actually, I also need to find something for my dad. He's a coffee enthusiast and loves cooking. Budget is flexible, maybe up to $100?"
)

print(response)


In [ ]:
# Memory-aware follow-up
response = agent.chat(
    user_id=USER_ID,
    message="What was the most budget-friendly option you suggested for Sarah again? I'm trying to decide between your recommendations."
)

print(response)


# Ask for a summary of all gift searches

In [ ]:
response = agent.chat(
    user_id=USER_ID,
    message="Can you give me a quick summary of all the gift ideas we've discussed today? I want to make a shopping list."
)

print(response)

## 8. Demo: Eco-Conscious Gift Finding

Test the agent's ability to find gifts matching specific values like sustainability.


# Search for eco-friendly gifts
response = agent.chat(
    user_id=USER_ID,
    message="One more thing — my sister is really into sustainability and zero-waste living. Can you find some eco-friendly gift options under $40?"
)

print(response)


## 9. End Session & Save Memory

# End the session to save short-term memory
agent.end_session(USER_ID)


## 10. Explore MongoDB Collections

View what's stored in MongoDB — products, agent metadata, tools, and user memories.


In [ ]:
# View agent metadata
print("=== Agent Metadata ===")
metadata = agent.get_metadata()
if metadata:
    print(f"Name: {metadata.get('name')}")
    print(f"LLM: {metadata.get('llm_provider')} / {metadata.get('llm_model')}")
    print(f"Tools: {metadata.get('tools', [])}")
    print(f"Created: {metadata.get('created_at')}")

# View registered tools
print("\n=== Registered Tools ===")
tools = agent.get_tools()
for tool in tools:
    print(f"  • {tool['name']}: {tool['description'][:60]}...")

# View user stats
print(f"\n=== User Memory Stats ({USER_ID}) ===")
stats = agent.get_user_stats(USER_ID)
print(f"  Conversations: {stats['conversation_count']}")
print(f"  Short-term memories: {stats['short_term_count']}")
print(f"  Long-term memories: {stats['long_term_count']}")

# View extracted memories
print(f"\n=== Extracted User Memories ===")
memories = agent.get_user_memories(USER_ID, limit=5)
for m in memories:
    print(f"  [{m.get('category', 'general')}] {m['content']}")


In [ ]:
# View product catalog stats
print("=== Product Catalog Stats ===")
print(f"Total products: {_products.count_documents({})}")

# Products by category
print("\nProducts by category:")
pipeline = [
    {"$group": {"_id": "$category", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
for cat in _products.aggregate(pipeline):
    print(f"  {cat['_id']}: {cat['count']}")

# Price range
print("\nPrice range:")
min_price = _products.find_one(sort=[("price", 1)])["price"]
max_price = _products.find_one(sort=[("price", -1)])["price"]
print(f"  ${min_price:.2f} - ${max_price:.2f}")
